# Exploring ERPGantry Shape file downloaded from LTA DataMall

https://datamall.lta.gov.sg/content/dam/datamall/datasets/Geospatial/ERPGantry.zip

Reading shapefile returns a linestring object

Key libraries:
* [GemGIS](https://gemgis.readthedocs.io/en/latest/index.html)
* [Geopandas](https://geopandas.org/en/stable/#)

In [26]:
import os
import sys
from shapely.geometry import shape
import geopandas as gpd
PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, PROJECT_PATH)

shapefile_path = os.path.join(PROJECT_PATH, "data", "ERPGantry_Jul2023", "Gantry.shp")
# Open the shapefile
gdf = gpd.read_file(rf"{shapefile_path}")


gdf_wgs84 = gdf.to_crs({"init": "epsg:4326"})

In [27]:
gdf_wgs84

,TYP_CD,TYP_CD_DES,geometry
0,None,ERP,"LINESTRING (103.79685 1.28683, 103.79675 1.28665)"
1,None,ERP,"LINESTRING (103.84436 1.27335, 103.84447 1.27326)"
2,None,ERP,"LINESTRING (103.87172 1.32013, 103.87180 1.32017)"
3,None,ERP,"LINESTRING (103.86461 1.32905, 103.86462 1.32896)"
4,None,ERP,"LINESTRING (103.86461 1.32912, 103.86461 1.32905)"
...,...,...,...
101,None,ERP,"LINESTRING (103.78782 1.35362, 103.78773 1.35348)"
102,None,ERP,"LINESTRING (103.85925 1.34654, 103.85952 1.34664)"
103,None,ERP,"LINESTRING (103.76090 1.31516, 103.76082 1.31517)"
104,None,ERP,"LINESTRING (103.87444 1.30712, 103.87445 1.30703)"


Using GemGIS to extract coordinates from

In [37]:
import gemgis as gg
gdf_wgs_xy = gg.vector.extract_xy(gdf=gdf_wgs84)
gdf_wgs_xy["Lat"], gdf_wgs_xy["Lon"] = gdf_wgs_xy["Y"], gdf_wgs_xy["X"]
gdf_wgs_xy

,TYP_CD,TYP_CD_DES,geometry,X,Y,Lat,Lon
0,None,ERP,POINT (103.79685 1.28683),103.80,1.29,1.29,103.80
1,None,ERP,POINT (103.79675 1.28665),103.80,1.29,1.29,103.80
2,None,ERP,POINT (103.84436 1.27335),103.84,1.27,1.27,103.84
3,None,ERP,POINT (103.84447 1.27326),103.84,1.27,1.27,103.84
4,None,ERP,POINT (103.87172 1.32013),103.87,1.32,1.32,103.87
...,...,...,...,...,...,...,...
207,None,ERP,POINT (103.76082 1.31517),103.76,1.32,1.32,103.76
208,None,ERP,POINT (103.87444 1.30712),103.87,1.31,1.31,103.87
209,None,ERP,POINT (103.87445 1.30703),103.87,1.31,1.31,103.87
210,None,ERP,POINT (103.84014 1.28037),103.84,1.28,1.28,103.84


Filter out alternate entries since they represent endpoints of LineString after extracting the start/end coordinates from it.

We assume that the start and end points are relatively close to each other (perhaps representing the span of the infrastructure). As such taking either start/end coordinates would not result in extremely off position of the required gantries geolocations

In [40]:
start_lat_lon_df = gdf_wgs_xy.loc[0::2][["TYP_CD_DES", "Lat", "Lon"]]
start_lat_lon_df

,TYP_CD_DES,Lat,Lon
0,ERP,1.29,103.80
2,ERP,1.27,103.84
4,ERP,1.32,103.87
6,ERP,1.33,103.86
8,ERP,1.33,103.86
...,...,...,...
202,ERP,1.35,103.79
204,ERP,1.35,103.86
206,ERP,1.32,103.76
208,ERP,1.31,103.87
